In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math

import sys

import numpy as np

import matplotlib.pyplot as plt

import keras

import pandas as pd

import numpy as np

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import LSTM

from keras.layers import Dropout

from keras.layers import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

In [3]:
%cd /content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator

/content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator


In [4]:
df=pd.read_csv("bitcoin_indicator.csv")
df_time = pd.read_csv("time.csv")
print('Number of rows and columns:', df.shape)

time_step = 30
index_rsi = 14
n_days = 15

df

Number of rows and columns: (3962, 25)


,close,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,vpt,...,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb,volume
0,3091000.0,14.944005,1.163482,3063000.0,3084200.0,3170200.0,3289500.0,3.215300e+06,24.735450,-50913.192669,...,53.117928,3.101464e+06,3.154992e+06,3.207830e+06,3.195372e+06,-2.058452e+09,3.610062e+06,3289500.0,2.968938e+06,5388.617973
1,3087000.0,14.942710,1.163482,3087000.0,3088600.0,3137500.0,3279000.0,3.216817e+06,25.546448,-50921.710773,...,54.027715,3.096642e+06,3.142630e+06,3.196323e+06,3.191819e+06,-2.356487e+09,3.612042e+06,3279000.0,2.945958e+06,6582.364759
2,3071000.0,14.937514,1.157451,3087000.0,3085200.0,3106200.0,3262700.0,3.216783e+06,25.304465,-50942.230512,...,52.648713,3.088095e+06,3.129606e+06,3.184387e+06,3.187857e+06,-1.445045e+09,3.603251e+06,3262700.0,2.922149e+06,3959.027066
3,3050000.0,14.930652,1.157451,3050000.0,3072400.0,3086100.0,3243350.0,3.215633e+06,26.044568,-50979.943691,...,53.324172,3.075397e+06,3.115132e+06,3.171588e+06,3.183338e+06,-1.897196e+09,3.586173e+06,3243350.0,2.900527e+06,5515.103573
4,3003000.0,14.915122,1.139615,3050000.0,3060400.0,3063700.0,3223000.0,3.214083e+06,25.442177,-51042.834596,...,54.476339,3.051264e+06,3.094745e+06,3.155532e+06,3.177425e+06,-1.473320e+09,3.572426e+06,3223000.0,2.873574e+06,4081.218313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,22320000.0,16.920994,19.407960,22320000.0,22490200.0,22593100.0,22691850.0,2.493500e+07,46.624136,2341.093255,...,53.458386,2.245848e+07,2.256368e+07,2.277992e+07,2.431527e+07,-2.116684e+08,2.316037e+07,22691850.0,2.222333e+07,1666.680241
3958,22403000.0,16.924705,19.480131,22320000.0,22430800.0,22571300.0,22662550.0,2.482275e+07,48.006214,2346.997775,...,53.335951,2.243999e+07,2.253446e+07,2.274402e+07,2.425257e+07,-1.222620e+08,2.312610e+07,22662550.0,2.219900e+07,1587.817986
3959,23004000.0,16.951179,19.480131,23004000.0,22496800.0,22612900.0,22660250.0,2.472663e+07,51.684312,2391.934249,...,43.927284,2.262799e+07,2.261983e+07,2.276878e+07,2.421164e+07,1.172543e+08,2.311609e+07,22660250.0,2.220441e+07,1675.061266
3960,22790000.0,16.941832,19.298913,23004000.0,22581400.0,22618800.0,22651250.0,2.462907e+07,47.797063,2377.924759,...,45.856043,2.268200e+07,2.265077e+07,2.277080e+07,2.416503e+07,-1.490895e+08,2.308805e+07,22651250.0,2.221445e+07,1505.954700


In [5]:
df_time = df_time.iloc[:,0]

In [6]:
df_time = df_time.str[:10]

In [7]:
res = []

# training set/ test set

In [8]:
feature = df.iloc[:,1:]

idx = int(feature.shape[0]*0.8)

training_set = feature.iloc[:idx].values
test_set = feature.iloc[idx:].values
training_set.shape

(3169, 24)

In [9]:
training_set

array([[1.49440052e+01, 1.16348160e+00, 3.06300000e+06, ...,
        3.28950000e+06, 2.96893767e+06, 5.38861797e+03],
       [1.49427103e+01, 1.16348160e+00, 3.08700000e+06, ...,
        3.27900000e+06, 2.94595788e+06, 6.58236476e+03],
       [1.49375138e+01, 1.15745125e+00, 3.08700000e+06, ...,
        3.26270000e+06, 2.92214868e+06, 3.95902707e+03],
       ...,
       [1.81156490e+01, 2.14693282e+01, 7.37100000e+07, ...,
        7.54120500e+07, 7.09528907e+07, 2.58543345e+03],
       [1.81241328e+01, 2.16522442e+01, 7.37100000e+07, ...,
        7.52044000e+07, 7.09676991e+07, 2.22946506e+03],
       [1.81121971e+01, 2.13953462e+01, 7.37100000e+07, ...,
        7.48318500e+07, 7.14908085e+07, 1.95423116e+03]])

In [10]:
# 정규화
ss = StandardScaler()

training_set_scaled = ss.fit_transform(training_set)

X_train = []

y_train = []

for i in range(time_step, idx-n_days):
    X_train.append(training_set_scaled[i-time_step:i])

    y_train.append(training_set_scaled[i+n_days, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
y_train

array([-1.66957522, -1.66488984, -1.67506461, ...,  2.15800037,
        2.16822929,  2.15383842])

In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_train.shape

(3124, 30, 24)

In [12]:
y_train.shape

(3124,)

# 모델 학습

In [13]:
model = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.4))


model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.4))

#model.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model.add(LSTM(units = 100, return_sequences = True))
model.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.4))


model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model.add(LSTM(units = 60))
model.add(Dropout(0.4))


# Adding the output layer i 
 
model.add(Dense(units = 1))

# Compiling the RNN

model.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 18s 21ms/step - loss: 0.1454
Epoch 2/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0580
Epoch 3/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0485
Epoch 4/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0434
Epoch 5/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0406
Epoch 6/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0388
Epoch 7/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0361
Epoch 8/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0336
Epoch 9/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0320
Epoch 10/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0302
Epoch 11/80
98/98 [==============================] - 2s 21ms/step - loss: 0.0291
Epoch 12/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0313
Epoch 13/80
98/98 [=================

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 100)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 30, 100)           80400     
                                                                 
 dropout (Dropout)           (None, 30, 100)           0         
                                                                 
 lstm_2 (LSTM)               (None, 30, 100)           80400     
                                                                 
 dropout_1 (Dropout)         (None, 30, 100)           0         
                                                                 
 lstm_3 (LSTM)               (None, 30, 100)           80400     
                                                                 
 dropout_2 (Dropout)         (None, 30, 100)           0

# 모델이 예상한 값 불러오기

In [15]:
dataset_train = feature.iloc[:idx] 

dataset_test = feature.iloc[idx:]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)

inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_step:].values

#inputs = inputs.reshape(-1,n_features)

inputs = ss.transform(inputs)

X_test = []
for i in range(time_step, dataset_test.shape[0]):

    X_test.append(inputs[i-time_step:i])

X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

print(X_test.shape)

(763, 30, 24)


In [16]:
predicted_stock_price = model.predict(X_test)

24/24 [==============================] - 3s 8ms/step


In [17]:
X_test.shape[0]

763

In [18]:
feature

,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,vpt,obv,...,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb,volume
0,14.944005,1.163482,3063000.0,3084200.0,3170200.0,3289500.0,3.215300e+06,24.735450,-50913.192669,7.758012e+04,...,53.117928,3.101464e+06,3.154992e+06,3.207830e+06,3.195372e+06,-2.058452e+09,3.610062e+06,3289500.0,2.968938e+06,5388.617973
1,14.942710,1.163482,3087000.0,3088600.0,3137500.0,3279000.0,3.216817e+06,25.546448,-50921.710773,7.099776e+04,...,54.027715,3.096642e+06,3.142630e+06,3.196323e+06,3.191819e+06,-2.356487e+09,3.612042e+06,3279000.0,2.945958e+06,6582.364759
2,14.937514,1.157451,3087000.0,3085200.0,3106200.0,3262700.0,3.216783e+06,25.304465,-50942.230512,6.703873e+04,...,52.648713,3.088095e+06,3.129606e+06,3.184387e+06,3.187857e+06,-1.445045e+09,3.603251e+06,3262700.0,2.922149e+06,3959.027066
3,14.930652,1.157451,3050000.0,3072400.0,3086100.0,3243350.0,3.215633e+06,26.044568,-50979.943691,6.152363e+04,...,53.324172,3.075397e+06,3.115132e+06,3.171588e+06,3.183338e+06,-1.897196e+09,3.586173e+06,3243350.0,2.900527e+06,5515.103573
4,14.915122,1.139615,3050000.0,3060400.0,3063700.0,3223000.0,3.214083e+06,25.442177,-51042.834596,5.744241e+04,...,54.476339,3.051264e+06,3.094745e+06,3.155532e+06,3.177425e+06,-1.473320e+09,3.572426e+06,3223000.0,2.873574e+06,4081.218313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,16.920994,19.407960,22320000.0,22490200.0,22593100.0,22691850.0,2.493500e+07,46.624136,2341.093255,1.475735e+06,...,53.458386,2.245848e+07,2.256368e+07,2.277992e+07,2.431527e+07,-2.116684e+08,2.316037e+07,22691850.0,2.222333e+07,1666.680241
3958,16.924705,19.480131,22320000.0,22430800.0,22571300.0,22662550.0,2.482275e+07,48.006214,2346.997775,1.477323e+06,...,53.335951,2.243999e+07,2.253446e+07,2.274402e+07,2.425257e+07,-1.222620e+08,2.312610e+07,22662550.0,2.219900e+07,1587.817986
3959,16.951179,19.480131,23004000.0,22496800.0,22612900.0,22660250.0,2.472663e+07,51.684312,2391.934249,1.478998e+06,...,43.927284,2.262799e+07,2.261983e+07,2.276878e+07,2.421164e+07,1.172543e+08,2.311609e+07,22660250.0,2.220441e+07,1675.061266
3960,16.941832,19.298913,23004000.0,22581400.0,22618800.0,22651250.0,2.462907e+07,47.797063,2377.924759,1.477492e+06,...,45.856043,2.268200e+07,2.265077e+07,2.277080e+07,2.416503e+07,-1.490895e+08,2.308805e+07,22651250.0,2.221445e+07,1505.954700


# 표준화했던 값을 원래 값으로 되돌림

In [19]:
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean
original.shape

(763, 1)

In [20]:
index = df.shape[0] - original.shape[0]
index

3199

In [21]:
y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
rmse

2.872630468159154

In [22]:
def measure_accuarcy(predict, real = dataset_test.values[:,0], n_days=1):
    predict_result = []
    real_result = []
    count = 0
    result = 0
    correct_up = 0
    correct_down = 0
    real_up = 0
    real_down = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==1:
            real_up += 1
            if predict_result[i] ==1:
                correct_up += 1
        elif real_result[i]==0:
            if predict_result[i] == 0:
                correct_down += 1
                
    count = correct_up + correct_down
    
    return (count/n_predicting_days,correct_up,correct_down)

In [23]:
res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))

In [24]:
def measure_diff(predict, real = dataset_test.values[:,0], n_days=1):
    
    predict_result = []
    real_result = []
    result = 0
    predict_diff = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==predict_result[i]:
            predict_diff += abs(predict[i]-real[i])
        else:
            predict_diff -= abs(predict[i]-real[i])
    
    return predict_diff

In [25]:
measure_diff(original, n_days = 15)

array([34.39019], dtype=float32)

In [26]:
model2 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model2.add(LSTM(100, return_sequences=True))
model2.add(Dropout(0.8))


model2.add(LSTM(100, return_sequences=True))
model2.add(Dropout(0.8))

#model2.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 100, return_sequences = True))
model2.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 60, return_sequences = True))
model2.add(Dropout(0.8))

model2.add(LSTM(units = 60, return_sequences = True))
model2.add(Dropout(0.8))


model2.add(LSTM(units = 60, return_sequences = True))
model2.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 60))
model2.add(Dropout(0.8))


# Adding the output layer i 
 
model2.add(Dense(units = 1))

# Compiling the RNN

model2.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model2.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 12s 21ms/step - loss: 0.2756
Epoch 2/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1570
Epoch 3/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1358
Epoch 4/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1377
Epoch 5/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1224
Epoch 6/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1088
Epoch 7/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1123
Epoch 8/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1131
Epoch 9/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1048
Epoch 10/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1054
Epoch 11/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0982
Epoch 12/80
98/98 [==============================] - 2s 21ms/step - loss: 0.1014
Epoch 13/80
98/98 [=================

In [27]:
predicted_stock_price = model2.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 3s 8ms/step
3.0091192785289986
[17.634342]


In [28]:
model3 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model3.add(LSTM(100, return_sequences=True))
model3.add(Dropout(0.4))


model3.add(LSTM(100, return_sequences=True))
model3.add(Dropout(0.4))

#model3.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))

model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))


model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100))
model3.add(Dropout(0.4))


# Adding the output layer i 
 
model3.add(Dense(units = 1))

# Compiling the RNN

model3.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model3.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 12s 22ms/step - loss: 0.1277
Epoch 2/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0499
Epoch 3/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0451
Epoch 4/80
98/98 [==============================] - 2s 23ms/step - loss: 0.0414
Epoch 5/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0328
Epoch 6/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0360
Epoch 7/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0320
Epoch 8/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0283
Epoch 9/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0293
Epoch 10/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0271
Epoch 11/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0243
Epoch 12/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0241
Epoch 13/80
98/98 [=================

In [29]:
predicted_stock_price = model3.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 3s 9ms/step
2.8822293102639898
[37.8522]


In [30]:
model4 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model4.add(LSTM(100, return_sequences=True))
model4.add(Dropout(0.8))


model4.add(LSTM(100, return_sequences=True))
model4.add(Dropout(0.8))

#model4.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))

model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))


model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100))
model4.add(Dropout(0.8))


# Adding the output layer i 
 
model4.add(Dense(units = 1))

# Compiling the RNN

model4.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model4.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 13s 22ms/step - loss: 0.2386
Epoch 2/80
98/98 [==============================] - 2s 22ms/step - loss: 0.1352
Epoch 3/80
98/98 [==============================] - 2s 22ms/step - loss: 0.1137
Epoch 4/80
98/98 [==============================] - 2s 22ms/step - loss: 0.1084
Epoch 5/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0975
Epoch 6/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0924
Epoch 7/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0933
Epoch 8/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0807
Epoch 9/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0864
Epoch 10/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0848
Epoch 11/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0795
Epoch 12/80
98/98 [==============================] - 2s 22ms/step - loss: 0.0792
Epoch 13/80
98/98 [=================

In [31]:
predicted_stock_price = model4.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 3s 9ms/step
2.9827079022698606
[42.959095]


In [32]:
model5 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model5.add(LSTM(100, return_sequences=True))
model5.add(Dropout(0.4))

#model5.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 100, return_sequences = True))
model5.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 60, return_sequences = True))
model5.add(Dropout(0.4))

model5.add(LSTM(units = 60, return_sequences = True))
model5.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 60))
model5.add(Dropout(0.4))

# Adding the output layer i 
 
model5.add(Dense(units = 1))

# Compiling the RNN

model5.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model5.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 10s 16ms/step - loss: 0.1248
Epoch 2/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0479
Epoch 3/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0460
Epoch 4/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0437
Epoch 5/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0382
Epoch 6/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0363
Epoch 7/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0343
Epoch 8/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0297
Epoch 9/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0305
Epoch 10/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0289
Epoch 11/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0252
Epoch 12/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0286
Epoch 13/80
98/98 [=================

In [33]:
predicted_stock_price = model5.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 7ms/step
2.824452723670035
[41.51143]


In [34]:
model6 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model6.add(LSTM(100, return_sequences=True))
model6.add(Dropout(0.8))

#model6.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 100, return_sequences = True))
model6.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 60, return_sequences = True))
model6.add(Dropout(0.8))

model6.add(LSTM(units = 60, return_sequences = True))
model6.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 60))
model6.add(Dropout(0.8))

# Adding the output layer i 
 
model6.add(Dense(units = 1))

# Compiling the RNN

model6.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model6.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 16ms/step - loss: 0.2600
Epoch 2/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1473
Epoch 3/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1372
Epoch 4/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1235
Epoch 5/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1086
Epoch 6/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1085
Epoch 7/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1077
Epoch 8/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1101
Epoch 9/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1088
Epoch 10/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1055
Epoch 11/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0949
Epoch 12/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0926
Epoch 13/80
98/98 [==================

In [35]:
predicted_stock_price = model6.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 7ms/step
2.9228026100212987
[38.034767]


In [36]:
model7 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model7.add(LSTM(100, return_sequences=True))
model7.add(Dropout(0.4))

#model7.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100, return_sequences = True))
model7.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100, return_sequences = True))
model7.add(Dropout(0.4))

model7.add(LSTM(units = 100, return_sequences = True))
model7.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100))
model7.add(Dropout(0.4))

# Adding the output layer i 
 
model7.add(Dense(units = 1))

# Compiling the RNN

model7.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model7.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 10s 17ms/step - loss: 0.0928
Epoch 2/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0481
Epoch 3/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0405
Epoch 4/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0389
Epoch 5/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0335
Epoch 6/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0314
Epoch 7/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0280
Epoch 8/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0296
Epoch 9/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0265
Epoch 10/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0249
Epoch 11/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0247
Epoch 12/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0238
Epoch 13/80
98/98 [=================

In [37]:
predicted_stock_price = model7.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 7ms/step
2.87541207538877
[14.611294]


In [38]:
model8 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model8.add(LSTM(100, return_sequences=True))
model8.add(Dropout(0.8))

#model8.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100, return_sequences = True))
model8.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100, return_sequences = True))
model8.add(Dropout(0.8))

model8.add(LSTM(units = 100, return_sequences = True))
model8.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100))
model8.add(Dropout(0.8))

# Adding the output layer i 
 
model8.add(Dense(units = 1))

# Compiling the RNN

model8.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model8.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 17ms/step - loss: 0.2158
Epoch 2/80
98/98 [==============================] - 2s 17ms/step - loss: 0.1210
Epoch 3/80
98/98 [==============================] - 2s 17ms/step - loss: 0.1125
Epoch 4/80
98/98 [==============================] - 2s 17ms/step - loss: 0.1007
Epoch 5/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0919
Epoch 6/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0893
Epoch 7/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0781
Epoch 8/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0851
Epoch 9/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0751
Epoch 10/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0692
Epoch 11/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0738
Epoch 12/80
98/98 [==============================] - 2s 17ms/step - loss: 0.0784
Epoch 13/80
98/98 [==================

In [39]:
predicted_stock_price = model8.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 7ms/step
2.951158934844124
[19.767517]


In [40]:
model9 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model9.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model9.add(LSTM(100, return_sequences=True))
model9.add(Dropout(0.4))


model9.add(LSTM(60, return_sequences=True))
model9.add(Dropout(0.4))

#model9.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model9.add(LSTM(60))
model9.add(Dropout(0.4))

# Adding the output layer i 
 
model9.add(Dense(units = 1))

# Compiling the RNN

model9.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model9.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 6s 11ms/step - loss: 0.1039
Epoch 2/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0474
Epoch 3/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0398
Epoch 4/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0401
Epoch 5/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0350
Epoch 6/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0369
Epoch 7/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0298
Epoch 8/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0285
Epoch 9/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0298
Epoch 10/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0291
Epoch 11/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0277
Epoch 12/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0291
Epoch 13/80
98/98 [==================

In [41]:
predicted_stock_price = model9.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.8747133317037052
[25.412144]


In [42]:
model10 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model10.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model10.add(LSTM(100, return_sequences=True))
model10.add(Dropout(0.8))


model10.add(LSTM(60, return_sequences=True))
model10.add(Dropout(0.8))

#model10.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model10.add(LSTM(60))
model10.add(Dropout(0.8))

# Adding the output layer i 
 
model10.add(Dense(units = 1))

# Compiling the RNN

model10.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model10.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 7s 11ms/step - loss: 0.2346
Epoch 2/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1410
Epoch 3/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1354
Epoch 4/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1070
Epoch 5/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1123
Epoch 6/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1026
Epoch 7/80
98/98 [==============================] - 1s 12ms/step - loss: 0.1035
Epoch 8/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0973
Epoch 9/80
98/98 [==============================] - 1s 12ms/step - loss: 0.1055
Epoch 10/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1019
Epoch 11/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0965
Epoch 12/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0892
Epoch 13/80
98/98 [==================

In [43]:
predicted_stock_price = model10.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.9082876088598715
[19.69855]


In [44]:
model11 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model11.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model11.add(LSTM(100, return_sequences=True))
model11.add(Dropout(0.4))


model11.add(LSTM(100, return_sequences=True))
model11.add(Dropout(0.4))

#model11.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model11.add(LSTM(100))
model11.add(Dropout(0.4))

# Adding the output layer i 
 
model11.add(Dense(units = 1))

# Compiling the RNN

model11.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model11.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 6s 12ms/step - loss: 0.0816
Epoch 2/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0457
Epoch 3/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0407
Epoch 4/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0325
Epoch 5/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0324
Epoch 6/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0285
Epoch 7/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0286
Epoch 8/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0256
Epoch 9/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0266
Epoch 10/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0247
Epoch 11/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0230
Epoch 12/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0227
Epoch 13/80
98/98 [==================

In [45]:
predicted_stock_price = model11.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.920896742539834
[33.73442]


In [46]:
model12 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model12.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model12.add(LSTM(100, return_sequences=True))
model12.add(Dropout(0.8))


model12.add(LSTM(100, return_sequences=True))
model12.add(Dropout(0.8))

#model12.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model12.add(LSTM(100))
model12.add(Dropout(0.8))

# Adding the output layer i 
 
model12.add(Dense(units = 1))

# Compiling the RNN

model12.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model12.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 6s 12ms/step - loss: 0.2071
Epoch 2/80
98/98 [==============================] - 1s 12ms/step - loss: 0.1150
Epoch 3/80
98/98 [==============================] - 1s 12ms/step - loss: 0.1025
Epoch 4/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0833
Epoch 5/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0784
Epoch 6/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0861
Epoch 7/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0797
Epoch 8/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0703
Epoch 9/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0738
Epoch 10/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0729
Epoch 11/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0756
Epoch 12/80
98/98 [==============================] - 1s 12ms/step - loss: 0.0774
Epoch 13/80
98/98 [==================

In [47]:
predicted_stock_price = model12.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.9447715184333645
[25.684278]


In [48]:
model13 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model13.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model13.add(LSTM(60))
model13.add(Dropout(0.4))


# Adding the output layer i 
 
model13.add(Dense(units = 1))

# Compiling the RNN

model13.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model13.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.0875
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0397
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0364
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0332
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0313
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0302
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0288
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0275
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0277
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0253
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0236
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0239
Epoch 13/80
98/98 [==============================

In [49]:
predicted_stock_price = model13.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.888144232218392
[53.619232]


In [50]:
model14 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model14.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model14.add(LSTM(60))
model14.add(Dropout(0.8))


# Adding the output layer i 
 
model14.add(Dense(units = 1))

# Compiling the RNN

model14.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model14.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2107
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1175
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1024
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1036
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0934
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0904
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0946
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0916
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0880
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0852
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0886
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0826
Epoch 13/80
98/98 [==============================

In [51]:
predicted_stock_price = model14.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.966072589624404
[20.362581]


In [52]:
model15 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model15.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model15.add(LSTM(100))
model15.add(Dropout(0.4))


# Adding the output layer i 
 
model15.add(Dense(units = 1))

# Compiling the RNN

model15.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model15.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 4s 7ms/step - loss: 0.0829
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0380
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0316
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0298
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0267
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0263
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0244
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0233
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0223
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0245
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0224
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0198
Epoch 13/80
98/98 [==============================

In [53]:
predicted_stock_price = model15.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.8679730079821377
[47.919365]


In [54]:
model16 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model16.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model16.add(LSTM(100))
model16.add(Dropout(0.8))


# Adding the output layer i 
 
model16.add(Dense(units = 1))

# Compiling the RNN

model16.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model16.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.1757
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0907
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0809
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0753
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0744
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0745
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0680
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0678
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0659
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0647
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0669
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0641
Epoch 13/80
98/98 [==============================

In [55]:
predicted_stock_price = model16.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9111831912646795
[50.94186]


In [56]:
model17 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model17.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model17.add(LSTM(60))
model17.add(Dropout(0.8))


# Adding the output layer i 
 
model17.add(Dense(units = 1))

# Compiling the RNN

model17.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model17.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2146
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1122
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1006
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0928
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0988
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0960
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0889
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0854
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0900
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0934
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0884
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0899
Epoch 13/80
98/98 [==============================

In [57]:
predicted_stock_price = model17.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9848400523950134
[31.020945]


In [58]:
model18 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model18.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model18.add(LSTM(40))
model18.add(Dropout(0.8))


# Adding the output layer i 
 
model18.add(Dense(units = 1))

# Compiling the RNN

model18.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model18.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2541
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1508
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1366
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1318
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1276
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1207
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1122
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1207
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1143
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1236
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1089
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1099
Epoch 13/80
98/98 [==============================

In [59]:
predicted_stock_price = model18.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
3.0294778336414026
[33.910713]


In [60]:
model19 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model19.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model19.add(LSTM(60))
model19.add(Dropout(0.8))


# Adding the output layer i 
 
model19.add(Dense(units = 1))

# Compiling the RNN

model19.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model19.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2361
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1243
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1050
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0975
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0922
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0927
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0890
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0877
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0855
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0893
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0828
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0865
Epoch 13/80
98/98 [==============================

In [61]:
predicted_stock_price = model19.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.946223510827274
[12.816397]


In [62]:
model20 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model20.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model20.add(LSTM(40))
model20.add(Dropout(0.8))


# Adding the output layer i 
 
model20.add(Dense(units = 1))

# Compiling the RNN

model20.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model20.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2763
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1591
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1361
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1226
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1223
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1202
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1178
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1142
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1120
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1158
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1180
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1086
Epoch 13/80
98/98 [==============================

In [63]:
predicted_stock_price = model20.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
3.0152580051100926
[33.43798]


In [64]:
model21 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model21.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[-2:])))

model21.add(LSTM(50))
model21.add(Dropout(0.8))


# Adding the output layer i 
 
model21.add(Dense(units = 1))

# Compiling the RNN

model21.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model21.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2602
Epoch 2/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1420
Epoch 3/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1175
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1115
Epoch 5/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1031
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1051
Epoch 7/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1060
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0993
Epoch 9/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1019
Epoch 10/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1049
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1003
Epoch 12/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1006
Epoch 13/80
98/98 [==============================

In [65]:
predicted_stock_price = model21.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9805487542630287
[40.402023]


In [66]:
score = pd.DataFrame(res)
score

,0
0,60.43
1,61.23
2,59.63
3,69.12
4,59.36
5,68.98
6,55.48
7,67.25
8,58.69
9,65.78


In [67]:
score.to_csv('res_3.csv', index=False)